In [6]:
import sys
import math
import pickle
import numpy as np

k = 10
sys.setrecursionlimit(1500)

In [7]:
def inputDigit(name="digitdata/trainingimages"):
    with open(name) as digitTxt:
        image = [list(line)[0:28] for line in digitTxt]
        rows = len(image)
    return image

In [8]:
digitImage = inputDigit()
print(len(digitImage))
print(len(digitImage[139999]))

140000
28


In [9]:
import collections

def readRough(name="digitdata/traininglabels"):
    with open(name) as label:
        labels = []
        for line in label:
            labels.append(line[0])
    return labels


In [10]:
label = readRough()
print (len(label))
#print sorted(collections.Counter(label).items())
class_ = sorted(collections.Counter(label).items())
print (class_)
with open('labelstats.txt', 'wb') as fp:
    pickle.dump(class_, fp)

5000
[('0', 479), ('1', 563), ('2', 488), ('3', 493), ('4', 535), ('5', 434), ('6', 501), ('7', 550), ('8', 462), ('9', 495)]


In [11]:
with open ('labelstats.txt', 'rb') as fp:
    class_ = pickle.load(fp)

digitImage = inputDigit()
training0=np.zeros(shape=(10, 28, 28))
training1=np.zeros(shape=(10, 28, 28))

count = 0
for i in range(5000):
    for row in range(28):
        for col in range(28):
            if digitImage[i*28 + row][col] == ' ':
                training1[int(label[i])][row][col] += 0
                training0[int(label[i])][row][col] += 1
            else:
                training1[int(label[i])][row][col] += 1
                training0[int(label[i])][row][col] += 0

for i in range(10):
    training1[i] = (training1[i] + k) / (class_[i][1] + k * 2)
    training0[i] = (training0[i] + k) / (class_[i][1] + k * 2)
print(training0[0][0])
print(training1[0][0])



[ 0.97995992  0.97995992  0.97995992  0.97995992  0.97995992  0.97995992
  0.97995992  0.97995992  0.97995992  0.97995992  0.97995992  0.97995992
  0.97995992  0.97995992  0.97995992  0.97995992  0.97995992  0.97995992
  0.97995992  0.97995992  0.97995992  0.97995992  0.97995992  0.97995992
  0.97995992  0.97995992  0.97995992  0.97995992]
[ 0.02004008  0.02004008  0.02004008  0.02004008  0.02004008  0.02004008
  0.02004008  0.02004008  0.02004008  0.02004008  0.02004008  0.02004008
  0.02004008  0.02004008  0.02004008  0.02004008  0.02004008  0.02004008
  0.02004008  0.02004008  0.02004008  0.02004008  0.02004008  0.02004008
  0.02004008  0.02004008  0.02004008  0.02004008]


In [12]:
# log P(class) + log P(f1,1 | class) + log P(f1,2 | class) + ... + log P(f28,28 | class)

test_rough = inputDigit(name = "digitdata/testimages")

answer = np.zeros(1000)
for i in range(1000):
    test_image = np.zeros(shape=(28,28))
    for row in range(28):
        for col in range(28):
            if test_rough[i*28+row][col] == ' ':
                test_image[row][col] = 0
            else:
                test_image[row][col] = 1
             
    posteriori = np.zeros(10)
    for class_num in range(10):
        posteriori[class_num] = math.log(class_[class_num][1])
        for row in range(28):
            for col in range(28):
                if test_image[row][col] == 0:
                    posteriori[class_num] += training0[class_num][row][col]
                else:
                    posteriori[class_num] += training1[class_num][row][col]    
    answer[i] = np.argmax(posteriori)
    
print(answer)



[ 1.  6.  1.  3.  1.  9.  7.  1.  1.  0.  4.  1.  9.  9.  1.  1.  2.  4.
  8.  1.  1.  7.  9.  4.  1.  1.  1.  1.  3.  1.  1.  1.  6.  1.  0.  6.
  1.  1.  1.  1.  1.  5.  3.  4.  1.  1.  1.  0.  9.  2.  1.  4.  9.  2.
  1.  7.  1.  4.  9.  4.  4.  0.  9.  9.  1.  8.  1.  1.  1.  3.  0.  7.
  1.  7.  8.  1.  1.  9.  6.  6.  4.  9.  1.  1.  0.  6.  9.  1.  9.  6.
  9.  1.  1.  8.  6.  3.  9.  1.  3.  1.  7.  9.  5.  9.  7.  9.  6.  3.
  6.  3.  7.  4.  6.  5.  8.  9.  9.  1.  8.  8.  1.  6.  1.  6.  1.  9.
  7.  1.  7.  1.  9.  5.  2.  7.  3.  1.  1.  1.  1.  1.  4.  7.  9.  7.
  3.  4.  1.  4.  0.  9.  3.  1.  9.  1.  0.  1.  1.  1.  4.  1.  4.  6.
  6.  4.  9.  9.  3.  4.  1.  9.  9.  7.  3.  9.  1.  7.  3.  1.  4.  1.
  1.  1.  6.  9.  1.  9.  7.  1.  0.  1.  3.  4.  7.  1.  9.  0.  1.  6.
  4.  6.  1.  1.  8.  0.  1.  3.  1.  2.  1.  1.  2.  1.  1.  1.  1.  1.
  1.  1.  0.  1.  0.  9.  4.  0.  1.  8.  1.  1.  9.  1.  4.  0.  1.  6.
  3.  3.  1.  1.  1.  1.  1.  1.  1.  6.  7.  2.  0

In [23]:
testlabels = readRough("digitdata/testlabels")


In [25]:
def confusion_matrix():
    num_each_class = [0 for x in range(10)]
    conf_matrix = [[0 for x in range(10)] for y in range(10)]
    for i in range(1000):
        conf_matrix[int(testlabels[i])][int(answer[i])] += 1
        num_each_class[int(testlabels[i])] += 1
            
    for i in range(10):
        for j in range(10):
            conf_matrix[i][j] /= num_each_class[i]
    
    return conf_matrix

In [27]:
conf_matrix = confusion_matrix()
for row in conf_matrix:
    print(row)

[0.7777777777777778, 0.0, 0.0, 0.011111111111111112, 0.011111111111111112, 0.011111111111111112, 0.12222222222222222, 0.0, 0.022222222222222223, 0.044444444444444446]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.019417475728155338, 0.5825242718446602, 0.24271844660194175, 0.02912621359223301, 0.009708737864077669, 0.0, 0.05825242718446602, 0.038834951456310676, 0.019417475728155338, 0.0]
[0.0, 0.42, 0.0, 0.49, 0.0, 0.0, 0.01, 0.06, 0.01, 0.01]
[0.0, 0.18691588785046728, 0.0, 0.0, 0.5981308411214953, 0.0, 0.028037383177570093, 0.0, 0.0, 0.18691588785046728]
[0.03260869565217391, 0.34782608695652173, 0.0, 0.22826086956521738, 0.043478260869565216, 0.09782608695652174, 0.043478260869565216, 0.03260869565217391, 0.03260869565217391, 0.14130434782608695]
[0.0, 0.3076923076923077, 0.0, 0.0, 0.06593406593406594, 0.0, 0.6263736263736264, 0.0, 0.0, 0.0]
[0.0, 0.32075471698113206, 0.0, 0.0, 0.018867924528301886, 0.0, 0.0, 0.5660377358490566, 0.009433962264150943, 0.08490566037735849]
[0